# Notes

Markov chain is a type of probability model that models the probability of random transitions between states. Usually in a Markov chain model, we assume the phenomenon we are modelling only have a few states, then we try to model the probability of any state proceeds to another state. In general, there are two types of Markov chain, discrete and continuous.

```Markov Property```

Both types of Markov chain obeys the Markov property:

**_The probability of the next state occurs only depends on the probability of the current state, but not the second, third, fourth... past states._**

### 1. Discrete-Time Markov Chain

Discrete-time Markov chain (DTMC) predicts the probability of transition between states directly by its transition matrix. Note that transition happens after a each fixed time interval (e.g. a second, a day, a month etc.).

```Example 1 (Simple Weather Model)```

Suppose there are three states, sunny, rainy and cloudy. The number in the diagram is the probability
of each day with the corresponding state changes to the other state. We define the time interval to a day, which means the weather will change on each day.

<img src='https://drive.google.com/uc?id=1nkw0b4MBLu4rfAsYU9hNPHTzqNxgH02e'>

The transition matrix of the model is

$$\left[\begin{matrix}
0.8 & 0.1 & 0.1 \\
0.2 & 0.5 & 0.3 \\
0.4 & 0.2 & 0.4
\end{matrix}\right].$$

Note that the row sum of the matrix should be 1. We may compute the stationary state vector by the following codes.

In [ ]:
import numpy as np
from numpy.linalg import matrix_power, inv

In [ ]:
P = np.array([[0.8, 0.1, 0.1], [0.2, 0.5, 0.3], [0.4, 0.2, 0.4]])
P

array([[0.8, 0.1, 0.1],
       [0.2, 0.5, 0.3],
       [0.4, 0.2, 0.4]])

In [ ]:
# initial state vector, representing the probability of "day 0 is sunny" is 100%
# the vector stands for the probability of [Sunny, Rainy, Cloudy]
u = np.array([1, 0, 0])
u

array([1, 0, 0])

In [ ]:
u@P    # day 1

array([0.8, 0.1, 0.1])

In [ ]:
u@P@P  # day 2

array([0.7 , 0.15, 0.15])

In [ ]:
u@matrix_power(P, 2)   # another command to compute matrix power

array([0.7 , 0.15, 0.15])

In [ ]:
u@matrix_power(P, 10000)    # estimate the stationary state vector

array([0.6, 0.2, 0.2])

### 2. Continuous-Time Markov Chain

Continuous-time Markov chain (CTMC) estimates the required time until next transition of states. Unlike DTMC, CTMC assumes the change of states occurs at any time but not after a fixed time interval. Hence, we may use CTMC to model stock price, interest rate changes, call arrivals in telecommunication systems, network congestion, genetic mutations, cellular chemical process etc.

We will represent a CTMC model by a rate matrix.

```Example 2 (Computer System)```

**Problem**

A computer center has two computers operating continuously. When one of them stops, the average time required to restart the process is exponentially distributed with a mean of 0.6 hours. The average estimated time to stop is also exponentially distributed with a mean of 0.4 hours.

**Solution**

Let $E_0$, $E_1$ and $E_2$ be the states "both computers are operating", "one computer is stopped" and "both computers are stopped". Note that the average time required to restart and to stop are obeying exponential distribution with $1/\lambda_1 = 0.6$ and $1/\lambda_2 = 0.4$ respectively. We need to compute the rate of a computer stops and restarts.

rate of a computer stops $= \lambda_1 = 1/0.6 = 1.66$ computer/hr

rate of a computer restarts $= \lambda_2 = 1/0.4 = 2.5$ computer/hr

Then from $E_0$ to $E_1$ and from $E_1$ to $E_2$, we have rate 1.66. From $E_2$ to $E_1$ and from $E_1$ to $E_0$, we have rate 2.5. Hence the rate matrix of the CTMC model is
$$
\left[\begin{matrix}
-1.66 & 1.66 & 0 \\
2.5 & -4.16 & 1.66 \\
0 & 2.5 & -2.5
\end{matrix}\right].
$$
Note that the row sum of a rate matrix is 0 but not 1. The diagonal entries are defined by the negative value of the other elements in that row.

We may find the stationary states vector by solving the equation $uQ = 0$, while the row sum of $u$ is 1. We may treat it as a linear programming problem.

In [ ]:
from scipy.optimize import linprog
obj_coeff = np.zeros(3)
con_coeff = np.array([
    [-1.66, 2.5, 0],
    [1.66, -4.16, 2.5],
    [0, 1.66, -2.5],
    [1, 1, 1]
])
con_bd = np.array([0, 0, 0, 1])
v = linprog(obj_coeff, A_eq = con_coeff, b_eq = con_bd, method="highs")

In [ ]:
if v.success:
    u1, u2, u3 = v.x
    print("Solution:")
    print(f"u1 = {u1:.4f}")
    print(f"u2 = {u2:.4f}")
    print(f"u3 = {u3:.4f}")
else:
    print("Optimization failed:", v.message)

Solution:
u1 = 0.4751
u2 = 0.3155
u3 = 0.2095


Hence, the stationary state vector is $u = \left[\begin{matrix}0.4751 & 0.3155 & 0.2095\end{matrix}\right]$. That means in long run, both computers will stop operation for 20.95% of time.

# Exercise

1. Suppose in a small town there are only two newspaper, A and B. Assume the population reading the two newspaper will not change. By statistics, 80% of the people who read newspaper A today will keep reading A tomorrow while 60% of the people who read newspaper B today will keep reading B tomorrow. Now we know on 5 June there are 60% of the population reads newspaper A and 40% reads newspaper B.

(a). Find the ratio of the population who read newspaper A and B on 3 June.

In [ ]:
M = np.array([[0.8, 0.2], [0.4, 0.6]])
M

array([[0.8, 0.2],
       [0.4, 0.6]])

In [ ]:
p = np.array([0.6, 0.4])

In [ ]:
inv_M = inv(M)

In [ ]:
p@inv_M@inv_M            # 25% of people read A and 75% of people read B

array([0.25, 0.75])

(b). Consider from 5 June, will the population of reading A and B converges to a constant ratio?

In [ ]:
p@matrix_power(M, 100)

array([0.66666667, 0.33333333])

In [ ]:
p@matrix_power(M, 1000)

array([0.66666667, 0.33333333])

In [ ]:
p@matrix_power(M, 10000)

array([0.66666667, 0.33333333])

In [ ]:
# We see that after 100, 1000 and 10000 days, the ratio of reading A and B converges to 2/3 and
# 1/3 respectively.

2. A man has two bag, denote them by A and B. There are two 10-dollar coins in bag A while three 5-dollar coins are in bag B. We define the states of bag A by $A_1$, $A_2$ and $A_3$, representing "two 10_dollar coins", "one 10_dollar coin" and "no 10-dollar coins". The man pick one coin from each bag at the same time and put them in the other bag. This process is repeated five times.

(a). Find the probability of $A_1$.

In [ ]:
N = np.array([[0, 1, 0], [1/6, 1/2, 1/3], [0, 2/3, 1/3]])
N

array([[0.        , 1.        , 0.        ],
       [0.16666667, 0.5       , 0.33333333],
       [0.        , 0.66666667, 0.33333333]])

In [ ]:
q = np.array([1, 0, 0])

In [ ]:
a5 = q@matrix_power(N, 5)
a5

array([0.09799383, 0.60416667, 0.29783951])

In [ ]:
# A_1 = 9.80%

(b). Find the expected value in bag A.

In [ ]:
a5[0]*(10+10) + a5[1]*(10+5) + a5[2]*(5+5)

14.000771604938269

3. Consider a machine that works for an exponential amount of time having mean 2 before breaking down; and suppose that it takes an exponential amount of time having mean 0.3 to repair the machine. If the machine is in working condition at $t = 0$, find the stationary state vector of the scenario.

In [ ]:
obj_coeff = np.zeros(2)
con_coeff = np.array([
    [-1/2, 1/2],
    [1/0.3, -1/0.3],
    [1, 1]
])
con_bd = np.array([0, 0, 1])
results = linprog(obj_coeff, A_eq = con_coeff, b_eq = con_bd, method="highs")

In [ ]:
if results.success:
    u1, u2 = results.x
    print("Solution:")
    print(f"u1 = {u1:.4f}")
    print(f"u2 = {u2:.4f}")
else:
    print("Optimization failed:", v.message)

Solution:
u1 = 0.5000
u2 = 0.5000
